In [11]:
'''Imbalanced training set
44k 0 
1.8k 1'''

'Imbalanced training set\n44k 0 \n1.8k 1'

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
train_df = pd.read_csv('TRAIN_FINAL.csv')

In [4]:
test_df = pd.read_csv('Test_v1.csv')

In [3]:
train_df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_360,col_361,col_362,col_363,col_364,col_365,col_366,col_367,col_368,TARGET
0,2,23,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56264.010000,0
1,2,51,510.0,897.72,2393.58,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
2,2,23,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83213.610000,0
3,2,23,120.0,59.67,59.67,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75297.300000,0
4,2,38,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176487.690000,0


In [10]:
train_df['TARGET'].value_counts()

0    43849
1     1763
Name: TARGET, dtype: int64

In [5]:
test_df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_360,col_361,col_362,col_363,col_364,col_365,col_366,col_367,col_368,index
0,2,23,0.0,0.00,0.00,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45444.060,0
1,2,23,0.0,0.00,0.00,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,208922.130,1
2,2,33,0.0,75.75,75.75,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60369.900,2
3,2,29,0.0,0.00,0.00,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979,3
4,2,30,0.0,47.13,47.13,0.0,0.0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979,4


In [9]:
X = train_df.drop(['TARGET'],axis=1)

In [13]:
y = train_df['TARGET']

In [23]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42,stratify = y)

In [61]:
import lightgbm as lgb 

lgb_params = {"objective" : "binary", "metric" : ['auc'],
              "num_leaves" : 8, "learning_rate" : 0.05, 
              "is_unbalance": True,
              "bagging_fraction" : 0.5, "feature_fraction" : 0.8, "bagging_frequency" : 9,
               }
    
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_val = lgb.Dataset(X_test, label=y_test)
model = lgb.train(lgb_params, lgb_train, 2000, valid_sets=[lgb_val], verbose_eval=100)

[100]	valid_0's auc: 0.831233
[200]	valid_0's auc: 0.827598
[300]	valid_0's auc: 0.822546
[400]	valid_0's auc: 0.818404
[500]	valid_0's auc: 0.814513
[600]	valid_0's auc: 0.810929
[700]	valid_0's auc: 0.808178
[800]	valid_0's auc: 0.805705
[900]	valid_0's auc: 0.803023
[1000]	valid_0's auc: 0.801162
[1100]	valid_0's auc: 0.79836
[1200]	valid_0's auc: 0.794689
[1300]	valid_0's auc: 0.790614
[1400]	valid_0's auc: 0.788102
[1500]	valid_0's auc: 0.786692
[1600]	valid_0's auc: 0.785187
[1700]	valid_0's auc: 0.783
[1800]	valid_0's auc: 0.781331
[1900]	valid_0's auc: 0.779774
[2000]	valid_0's auc: 0.777651


In [44]:
ind = test_df['index']
test = test_df.drop(['index'],axis=1)

In [30]:
pred_val = model.predict(test,num_iteration=model.best_iteration)

In [38]:
pred = [1 if i > 0.5 else 0 for i in pred_val]

In [39]:
y_lgbm = model.predict(test)

In [41]:
y_out = [1 if i > 0.5 else 0 for i in y_lgbm]

In [56]:
dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv","\t",index=False,columns=["index","TARGET"])

In [66]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500)
rf.fit(X_train,y_train)

pred = rf.predict(X_test)
print(roc_auc_score(y_test,pred))

0.510112758474


In [67]:
y_out = rf.predict(test)

In [68]:
dfdict={}

dfdict["index"]=ind
dfdict["TARGET"]=y_out

df=pd.DataFrame(dfdict)
df.to_csv("solution.csv","\t",index=False,columns=["index","TARGET"])